# Run Agent
- Implement a workflow to write a daily AI newsletter
- see README.md for details


In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable, Text
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url, AgentState

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            2.6.1
OpenAI Agents SDK  0.4.2
Pydantic           2.12.3


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# turn off open telemetry (langfuse)
os.environ["OTEL_SDK_DISABLED"] = "true"

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger
debuglevel = logging.DEBUG

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig()
    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(debuglevel)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(debuglevel)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(debuglevel)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.debug("Test debug message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

07:38:51 | NewsletterAgent.newsletter_agent | DEBUG | Test debug message
07:38:51 | NewsletterAgent.newsletter_agent | INFO | Test info message
07:38:51 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
07:38:51 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

In [5]:
# look up recent sessions
conn = sqlite3.connect("newsletter_agent.db")
AgentState.list_sessions(conn, n_records=10)


['test_newsletter_20251221183507320719',
 'test_newsletter_20251221183243388332',
 'test_newsletter_20251221182906785859',
 'test_newsletter_20251221182322841973',
 'test_newsletter_20251221081557459016',
 'test_newsletter_20251220184156049941',
 'test_newsletter_20251220183509149254',
 'test_newsletter_20251220104107302765',
 'test_newsletter_20251220101411525141',
 'test_newsletter_20251220100745970459']

In [6]:
z = AgentState.get_latest_by_session(conn, "test_newsletter_20251221183507320719")
z.step_name

'cluster_topics'

In [7]:
for y in z.get_all_by_session(conn, "test_newsletter_20251221183507320719"):
    print(y.session_id, y.step_name, y.updated_at)

test_newsletter_20251221183507320719 initialize 2025-12-21 18:35:07.339330
test_newsletter_20251221183507320719 gather_urls 2025-12-21 18:35:19.890720
test_newsletter_20251221183507320719 check_workflow_status 2025-12-21 18:35:21.004856
test_newsletter_20251221183507320719 filter_urls 2025-12-21 18:36:45.899320
test_newsletter_20251221183507320719 download_articles 2025-12-21 19:07:28.730751
test_newsletter_20251221183507320719 extract_summaries 2025-12-21 19:10:01.471944
test_newsletter_20251221183507320719 rate_articles 2025-12-21 19:15:38.448141
test_newsletter_20251221183507320719 cluster_topics 2025-12-21 19:19:02.331173


# Run Agent Worfklow

In [8]:
print("🚀 Creating NewsletterAgent...")
do_download=True
reprocess_since=None
# reprocess_since='2025-12-21 08:20:48.191939'
try:
    # set up state
    session_id = 'test_newsletter_20251221183507320719'
    step_name = 'filter_urls'
    del session_id
except Exception as e:
    print(e)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 reprocess_since=reprocess_since,
                                 verbose=False
                                )
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 reprocess_since=reprocess_since,
                                 verbose=False
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    agent.verbose=False  # there seems to be a bug where saved agents are saved or restored with verbose=True
    state.serialize_to_db("initialize")

🚀 Creating NewsletterAgent...
session_id is not defined
test_newsletter_20251222073851411839


In [9]:
agent.state.get_status()


{'headlines': {'total': 0},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'gather_urls',
  'workflow_complete': False,
  'progress_percentage': 0.0,
  'completed_steps': 0,
  'total_steps': 9,
  'max_edits': 2,
  'concurrency': 16,
  'do_download': True,
  'reprocess_since': None},
 'processing': {'topic_clusters': 0,
  'newsletter_sections': 0,
  'final_newsletter_length': 0}}

In [10]:
for step in agent.state.steps:
    print(step)
    print("---")


⚪ Gather URLs [not_started]
   
---
⚪ Filter URLs [not_started]
   
---
⚪ Download Articles [not_started]
   
---
⚪ Extract Summaries [not_started]
   
---
⚪ Rate Articles [not_started]
   
---
⚪ Cluster By Topic [not_started]
   
---
⚪ Select Sections [not_started]
   
---
⚪ Draft Sections [not_started]
   
---
⚪ Finalize Newsletter [not_started]
   
---


In [11]:
state.clear_errors()

In [12]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


07:38:54 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Starting check_workflow_status
07:38:54 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Completed check_workflow_status


⏱️  Total execution time: 7.50s
📊 Final result:
Here’s the current workflow status from persistent state:

- Progress: 0.0% (0/9 complete)
- Status summary: 0 complete, 0 started, 0 failed, 9 not started
- Next step: Step 0 — Gather URLs

Step-by-step state:
- Step 0: Gather URLs — not_started
- Step 1: Filter URLs — not_started
- Step 2: Download Articles — not_started
- Step 3: Extract Summaries — not_started
- Step 4: Rate Articles — not_started
- Step 5: Cluster By Topic — not_started
- Step 6: Select Sections — not_started
- Step 7: Draft Sections — not_started
- Step 8: Finalize Newsletter — not_started

Would you like me to:
- Start the workflow from the next step (Gather URLs), or
- Run all remaining steps to create the newsletter, or
- Run a specific step? (If so, tell me which step number/name.)

I’ll proceed once you tell me which action to take.


In [13]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


07:39:05 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Starting check_workflow_status
07:39:05 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Completed check_workflow_status
07:39:06 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Starting Step 1: Gather URLs
07:39:06 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Cleaning out download/sources: 
07:39:06 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Successfully cleaned out download/sources
07:39:06 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Cleaning out download/text: 
07:39:06 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Successfully cleaned out download/text
07:39:06 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Cleaning out download/html: 
07:39:06 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Successfully cleaned out download/html
2025-12-22 07:39:06,751 - fetcher_5630467

▶ Starting Step 1: gather_urls


2025-12-22 07:39:06,962 - fetcher_5630467920 - INFO - [fetch_rss] RSS fetch successful for HackerNoon: 50 articles
2025-12-22 07:39:06,963 - fetcher_5630467920 - INFO - [fetch_html] Fetching HTML from Techmeme: https://www.techmeme.com/river
2025-12-22 07:39:07,151 - fetcher_5630467920 - INFO - [fetch_rss] RSS fetch successful for Hacker News: 30 articles
2025-12-22 07:39:07,152 - fetcher_5630467920 - INFO - [fetch_rss] Fetching RSS from The Register: https://www.theregister.com/software/ai_ml/headlines.atom
2025-12-22 07:39:07,280 - fetcher_5630467920 - INFO - [fetch_rss] RSS fetch successful for The Register: 50 articles
2025-12-22 07:39:07,281 - fetcher_5630467920 - INFO - [fetch_html] Fetching HTML from The Verge: https://www.theverge.com/ai-artificial-intelligence
2025-12-22 07:39:08,680 - fetcher_5630467920 - INFO - [fetch_html] Source dict for Ars Technica: {'type': 'html', 'url': 'https://arstechnica.com/ai/', 'filename': 'Ars_Technica', 'include': ['^https://arstechnica.com/(\

,source,url
0,Ars Technica,28
1,Bloomberg,26
2,Business Insider,15
3,FT,49
4,Feedly AI,78
5,Hacker News,29
6,HackerNoon,50
7,New York Times,20
8,NewsAPI,41
9,Reddit,52


,source,title,url,published,rss_summary,id
0,Ars Technica,Google lobs lawsuit at search result scraping ...,https://arstechnica.com/google/2025/12/google-...,NaN,NaN,0
1,Ars Technica,LG TVs’ unremovable Copilot shortcut is the le...,https://arstechnica.com/gadgets/2025/12/lg-tvs...,NaN,NaN,1
2,Ars Technica,We asked four AI coding agents to rebuildMines...,https://arstechnica.com/ai/2025/12/the-ars-tec...,NaN,NaN,2
3,Ars Technica,YouTube bans two popular channels that created...,https://arstechnica.com/google/2025/12/youtube...,NaN,NaN,3
4,Ars Technica,OpenAI’s new ChatGPT image generator makes fak...,https://arstechnica.com/ai/2025/12/openais-new...,NaN,NaN,4
...,...,...,...,...,...,...
692,NewsAPI,Mayo Clinic study: Smartwatch AI algorithm ale...,https://www.postbulletin.com/health/mayo-clini...,2025-12-21T12:06:39Z,NaN,692
693,NewsAPI,"SIM introduces CareerSense, an AI-based career...",https://thesun.my/business/corporate-news/sim-...,2025-12-21T08:09:29Z,NaN,693
694,NewsAPI,Cardiovascular Health: New Risk‑Prediction Too...,https://www.zemeghub.com/2025/12/cardiovascula...,2025-12-21T10:56:36Z,NaN,694
695,NewsAPI,If San Jose becomes Bay Area’s data center cap...,https://www.mercurynews.com/2025/12/21/san-jos...,2025-12-21T12:01:57Z,NaN,695


07:40:03 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Completed Step 1: Gathered 838 articles
07:40:04 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Starting check_workflow_status
07:40:04 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Completed check_workflow_status


⏱️  Total execution time: 68.91s
📊 Final result:
Step 1 (Gather URLs) has been run.

Summary:
- Result: ✅ Step 1 gather_urls completed successfully
- Collected: 838 articles from 17 sources
- Articles stored in persistent state: 697
- Workflow progress: 11.1% (1/9 complete)
- Next step: Step 1 — Filter URLs

Would you like me to proceed to the next step (filter URLs), run a different step, or stop here?


In [14]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
10,Techmeme,154
4,Feedly AI,78
9,Reddit,52
12,The Register,50
6,HackerNoon,50
3,FT,49
8,NewsAPI,41
5,Hacker News,29
0,Ars Technica,28
1,Bloomberg,26


In [15]:
state.print_workflow_status()



📊 WORKFLOW STATUS
📈 Progress: 11.1% (1/9 complete)
➡️  Next step: Step 1: Filter URLs

📋 Step Details:
  ✅ Step 0: Gather URLs            | complete
      📝 Status: ✅ Step 1 gather_urls completed successfully! Gathered 838 articles from 17 sources.

📊 Articles stored in persistent state: 697
  ⭕ Step 1: Filter URLs            | not_started
  ⭕ Step 2: Download Articles      | not_started
  ⭕ Step 3: Extract Summaries      | not_started
  ⭕ Step 4: Rate Articles          | not_started
  ⭕ Step 5: Cluster By Topic       | not_started
  ⭕ Step 6: Select Sections        | not_started
  ⭕ Step 7: Draft Sections         | not_started
  ⭕ Step 8: Finalize Newsletter    | not_started



In [16]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)
# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


07:40:07 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Starting Step 2: Filter URLs
07:40:07 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | 🔍 Filtering 697 headlines...
07:40:07 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | 🔄 Checking for duplicates (all urls without date restrictions)
07:40:07 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | 🔍 Filtering 697 articles for dupes.
07:40:07 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | found with no cutoff set - ignoring as duplicate https://arstechnica.com/google/2025/12/google-lobs-lawsuit-at-search-result-scraping-firm-serpapi/
07:40:07 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | found with no cutoff set - ignoring as duplicate https://arstechnica.com/gadgets/2025/12/lg-tvs-unremovable-copilot-shortcut-is-the-least-of-smart-tvs-ai-problems/
07:40:07 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | found with no cutoff

▶ Starting Step 2: filter_urls


07:40:08 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | new - keeping https://arstechnica.com/features/2025/12/discworld-daleks-and-deep-13-a-geeky-holiday-tv-and-movie-watchlist/
07:40:08 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | found with no cutoff set - ignoring as duplicate https://arstechnica.com/science/2025/12/heres-why-nasa-and-the-space-force-are-interested-in-pizza-shaped-satellites/
07:40:08 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | new - keeping https://arstechnica.com/space/2025/12/trump-commits-to-moon-landing-by-2028-followed-by-a-lunar-outpost-two-years-later/
07:40:08 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | found with no cutoff set - ignoring as duplicate https://arstechnica.com/security/2025/12/browser-extensions-with-8-million-users-collect-extended-ai-conversations/
07:40:08 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | found with no cutoff set - ignoring as dupli

⏱️  Total execution time: 69.37s
📊 Final result:
✅ Step 2 filter_urls completed successfully! Removed 504 duplicate URLs, classified 193 new articles, found 75 AI-related.


In [17]:
headline_df = pd.DataFrame(state.headline_data)
headline_df


,source,title,url,published,rss_summary,id,isAI
0,Ars Technica,"Discworld, Daleks, and Deep 13: A geeky holida...",https://arstechnica.com/features/2025/12/discw...,NaN,NaN,7,False
1,Ars Technica,"Trump commits to Moon landing by 2028, followe...",https://arstechnica.com/space/2025/12/trump-co...,NaN,NaN,9,False
2,Bloomberg,"Nvidia Gains, Hospitals Hurt: Congress Winners...",https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,28,True
3,Bloomberg,Avishag Shaar-Yashuv for Bloomberg Businessweek,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,29,False
4,Bloomberg,That Time I Tried to Buy OpenAI,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,30,True
...,...,...,...,...,...,...,...
188,NewsAPI,Mayo Clinic study: Smartwatch AI algorithm ale...,https://www.postbulletin.com/health/mayo-clini...,2025-12-21T12:06:39Z,NaN,692,True
189,NewsAPI,"SIM introduces CareerSense, an AI-based career...",https://thesun.my/business/corporate-news/sim-...,2025-12-21T08:09:29Z,NaN,693,True
190,NewsAPI,Cardiovascular Health: New Risk‑Prediction Too...,https://www.zemeghub.com/2025/12/cardiovascula...,2025-12-21T10:56:36Z,NaN,694,True
191,NewsAPI,If San Jose becomes Bay Area’s data center cap...,https://www.mercurynews.com/2025/12/21/san-jos...,2025-12-21T12:01:57Z,NaN,695,False


In [18]:
state.clear_errors()

In [19]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


07:41:17 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Starting Step 3: Download Articles
07:41:17 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Starting concurrent scraping of 75 AI-related articles


▶ Starting Step 3: download_articles


07:41:17 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Launching browser for 75 URLs with 16 concurrent workers
07:41:17 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Worker 0 fetching 1 of 75 https://www.bloomberg.com/news/articles/2025-12-22/2025-winners-in-congress-include-nvidia-crypto-as-hospitals-lose
07:41:17 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Skipping ignored domain: www.bloomberg.com
07:41:17 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Worker 0 completed https://www.bloomberg.com/news/articles/2025-12-22/2025-winners-in-congress-include-nvidia-crypto-as-hospitals-lose with status: skipped
07:41:17 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Worker 0 fetching 2 of 75 https://www.wsj.com/business/entrepreneurship/teenage-founders-ecb9cbd3
07:41:17 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Skipping ignored domain: www.wsj.com
07:41:17 | NewsletterAgent.te

Starting with 74 rows...
Processing 74 files...
Reading and truncating files to 8192 tokens using text-embedding-3-large tokenizer...


Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Getting embeddings for 63 texts...


Creating indexed similarity matrix...
Finding pairs with similarity > 0.925...
Filtering dataframe...
  Pair: 52 vs 29
    52: HackerNoon - A New Benchmark Arms Race Is Redefining What “Good at AI” Even Means
    52: https://hackernoon.com/a-new-benchmark-arms-race-is-redefining-what-good-at-ai-even-means?source=rss
    29: Feedly AI - A New Benchmark Arms Race Is Redefining What “Good at AI” Even Means
    29: https://hackernoon.com/a-new-benchmark-arms-race-is-redefining-what-good-at-ai-even-means
  Pair: 50 vs 25
    50: HackerNoon - Why ISO Standards Are the Foundation of Responsible AI Governance
    50: https://hackernoon.com/why-iso-standards-are-the-foundation-of-responsible-ai-governance?source=rss
    25: Feedly AI - Why ISO Standards Are the Foundation of Responsible AI Governance
    25: https://hackernoon.com/why-iso-standards-are-the-foundation-of-responsible-ai-governance
Removing 2 rows due to high similarity 
Final dataframe has 72 rows (removed 2 rows)
Missing files:


,title,status,final_url
2,"Uber, Lyft Team Up With Baidu to Start Robotaxi Trials in UK",skipped,https://www.bloomberg.com/news/articles/2025-12-22/uber-teams-up-with-baidu-to-start-robotaxi-trials-in-uk
3,"HSBC's Sels on AI, Capex Cycle",skipped,https://www.bloomberg.com/news/videos/2025-12-22/hsbc-s-sels-on-ai-capex-cycle-video
48,Decompiling the Synergy: Human–LLM Teaming in Reverse Engineering [pdf],None,https://www.zionbasque.com/files/papers/dec-synergy-study.pdf
1,That Time I Tried to Buy OpenAI,skipped,https://www.bloomberg.com/news/articles/2025-12-22/how-to-buy-openai-before-its-ipo-one-investor-s-attempt
11,AI Startup FINNY Raises $17M for Advisor Prospecting Push,403,https://www.techrepublic.com/article/news-ai-startup-finny-funding/
62,These Teenagers Are Already Running Their Own AI Companies,skipped,https://www.wsj.com/business/entrepreneurship/teenage-founders-ecb9cbd3
18,"Chinese AI chipmaker Biren Technology plans to raise up to ~$623M in a Hong Kong IPO, pricing shares at ~$2.19 to ~$2.52, and expects trading to start January 2 (Sherry Qin/Wall Street Journal)",skipped,https://www.wsj.com/business/chinese-ai-chip-maker-biren-technology-plans-623-million-hong-kong-ipo-a5433f02
56,"Filings: Chinese AI startups Zhipu and MiniMax report 2024 revenue of $44.4M and $30.5M, respectively, as both prepare for Hong Kong IPOs in the coming weeks",skipped,https://www.bloomberg.com/news/articles/2025-12-22/china-s-openai-rivals-unveil-sales-details-ahead-of-stock-debuts
31,Why the McKinsey layoffs are a warning signal for consulting in the AI age,403,https://www.fastcompany.com/91463039/why-the-mckinsey-layoffs-are-a-warning-signal-for-consulting-in-the-ai-age-ai-layoffs-management-consulting
32,Inside HP’s AI bet to rebuild itself for the ‘work intelligence’ age,403,https://www.fastcompany.com/91462170/hp-ai-ces-work-intelligence


07:52:01 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Completed Step 3: Downloaded 64 articles


⏱️  Total execution time: 643.93s
📊 Final result:
✅ Step 3 download_articles completed successfully! Total articles: 193; AI-related articles: 72; HTML files: 61; Text files: 61


In [20]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["html_path"]==""])


,id,source,title,status,final_url
61,39,Bloomberg,"Uber, Lyft Team Up With Baidu to Start Robotaxi Trials in UK",skipped,https://www.bloomberg.com/news/articles/2025-12-22/uber-teams-up-with-baidu-to-start-robotaxi-trials-in-uk
62,42,Bloomberg,"HSBC's Sels on AI, Capex Cycle",skipped,https://www.bloomberg.com/news/videos/2025-12-22/hsbc-s-sels-on-ai-capex-cycle-video
63,214,Hacker News,Decompiling the Synergy: Human–LLM Teaming in Reverse Engineering [pdf],None,https://www.zionbasque.com/files/papers/dec-synergy-study.pdf
64,30,Bloomberg,That Time I Tried to Buy OpenAI,skipped,https://www.bloomberg.com/news/articles/2025-12-22/how-to-buy-openai-before-its-ipo-one-investor-s-attempt
65,134,Feedly AI,AI Startup FINNY Raises $17M for Advisor Prospecting Push,403,https://www.techrepublic.com/article/news-ai-startup-finny-funding/
66,590,WSJ,These Teenagers Are Already Running Their Own AI Companies,skipped,https://www.wsj.com/business/entrepreneurship/teenage-founders-ecb9cbd3
67,142,Feedly AI,"Chinese AI chipmaker Biren Technology plans to raise up to ~$623M in a Hong Kong IPO, pricing shares at ~$2.19 to ~$2.52, and expects trading to start January 2 (Sherry Qin/Wall Street Journal)",skipped,https://www.wsj.com/business/chinese-ai-chip-maker-biren-technology-plans-623-million-hong-kong-ipo-a5433f02
68,349,Techmeme,"Filings: Chinese AI startups Zhipu and MiniMax report 2024 revenue of $44.4M and $30.5M, respectively, as both prepare for Hong Kong IPOs in the coming weeks",skipped,https://www.bloomberg.com/news/articles/2025-12-22/china-s-openai-rivals-unveil-sales-details-ahead-of-stock-debuts
69,171,Feedly AI,Why the McKinsey layoffs are a warning signal for consulting in the AI age,403,https://www.fastcompany.com/91463039/why-the-mckinsey-layoffs-are-a-warning-signal-for-consulting-in-the-ai-age-ai-layoffs-management-consulting
70,172,Feedly AI,Inside HP’s AI bet to rebuild itself for the ‘work intelligence’ age,403,https://www.fastcompany.com/91462170/hp-ai-ces-work-intelligence


In [21]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

07:52:01 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Processing 72 AI articles for summarization
07:52:01 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Retrieved prompt 'newsagent/extract_summaries' from Langfuse
07:52:01 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Parsed prompt 'newsagent/extract_summaries': model=gpt-5-mini, reasoning_effort=medium, system_len=1273, user_len=43
07:52:01 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Using model 'gpt-5-mini' for summarization
07:52:01 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Starting summarization for 72 articles


▶ Starting Step 4: extract_summaries


07:52:37 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Extracting metadata from HTML files for 72 articles
07:52:40 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Retrieved prompt 'newsagent/item_distiller' from Langfuse
07:52:40 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Parsed prompt 'newsagent/item_distiller': model=gpt-5-mini, reasoning_effort=medium, system_len=1636, user_len=191


⏱️  Total execution time: 62.95s
📊 Final result:
✅ Step 4 extract_summaries completed successfully! Generated AI-powered summaries for 72/72 articles.
💾 Summaries stored in headline DataFrame.


In [22]:
# if we get a refusal, examine why and delete bad rows
bad_stuff =["Anthropic's Jack Cla",
           ]
print([len(s) for s in bad_stuff])
headline_df = state.headline_df
headline_df.loc[(headline_df["title"].str[:20].isin(bad_stuff))]



[20]


,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation,summary,description,tags,short_summary


In [23]:
# remove bad stuff
state.headline_df_to_dict(headline_df.loc[~headline_df["title"].str[:20].isin(bad_stuff)])


In [24]:
# User prompt to run workflow
# user_prompt = "Run step 5, Rate articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("rate_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


07:53:04 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Rating 72 AI articles using fn_rate_articles
07:53:04 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Calculating article ratings for 72 articles
07:53:04 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Rating recency
07:53:04 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Rating spam probability
07:53:04 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Retrieved prompt 'newsagent/rate_quality' from Langfuse
07:53:04 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, reasoning_effort=medium, system_len=1849, user_len=246


▶ Starting Step 5: rate_articles


07:53:06 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | low quality articles: {0.0: 55, 1.0: 6, 0.002472623024627213: 1, 0.43782350042481233: 1, 1.5125188821494485e-15: 1, 2.215948977336598e-08: 1, 2.222630272345391e-14: 1, 0.999999448776502: 1, 0.5621765025686553: 1, 0.9999998063873693: 1, 6.914400106940203e-13: 1, 2.172439935079017e-10: 1}
07:53:06 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Retrieved prompt 'newsagent/rate_on_topic' from Langfuse
07:53:06 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Parsed prompt 'newsagent/rate_on_topic': model=gpt-4.1, reasoning_effort=medium, system_len=1790, user_len=240
07:53:08 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | on topic articles: {1.0: 53, 0.9947799085613173: 1, 0.9999987335551229: 1, 0.029312225847906822: 1, 0.7310585784564795: 1, 0.0004305571096741222: 1, 0.9999945614411088: 1, 4.658886145103398e-15: 1, 0.9999993295729128: 1, 0.999933888949188: 1, 0.9998601

,id,site_name,title,bradley_terry,bt_z
26,26,The Washington Post,Chatbots can inflict harm. Why aren’t they held liable?A growing roster of suicides should draw attention to manipulative AI chatbots.,24.091328,1.48701
60,60,The Economic Times,"U.S. Stock market prediction: Can AI stocks Nvidia, Oracle bring Christmas holiday cheer for S&P 500, Nasdaq, Dow Jones on Monday? Key points to know",24.091328,1.48701
48,48,American Enterprise Institute,AI Chatbots Are Reshaping Political Persuasion,24.091328,1.48701
19,19,Reuters,"Sources: Nvidia plans to begin shipping its H200 chips to China before mid-February 2026 and expects initial shipments to be ~40,000 to 80,000 H200 units (Reuters)",24.091328,1.48701
16,16,Hacker Noon,"Is ""Agentic Programming"" the Next Big Shift?",24.091328,1.48701
...,...,...,...,...,...
52,52,Levelman,How AI Has Negatively Affected Asynchronous Learning,-24.091328,-1.48701
21,21,Engadget,Waymo vehicles are operating again in San Francisco following a power outage,-24.091328,-1.48701
59,59,Live Mint,Backlash to boom: Why brands are betting big on generative AI ads,-24.091328,-1.48701
31,31,The Wall Street Journal,These Teenagers Are Already Running Their Own AI Companies,-24.091328,-1.48701


07:53:20 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Top 10 ids: [26, 60, 48, 19, 16, 55, 42, 36, 6, 33]
07:53:20 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | After round 1/14: 
07:53:20 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Number of ranking changes: 71
07:53:20 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Sum of absolute ranking changes: 1685.0 (avg rank chg 23.73)
07:53:20 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | ---------------------------------------------------
07:53:20 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Running round 2 of max 14
07:53:20 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | ---------------------------------------------------
07:53:20 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Generated 12 battle batches
07:53:20 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Total stories in batche

...

,id,site_name,title,bradley_terry,bt_z
19,19,Reuters,"Sources: Nvidia plans to begin shipping its H200 chips to China before mid-February 2026 and expects initial shipments to be ~40,000 to 80,000 H200 units (Reuters)",61.831374,3.597285
55,55,Benzinga,OpenAI Doubles Compute Margins To Nearly 70% Amid AI Race: Report,52.833315,3.073787
57,57,Sky News,Fears grow of AI bubble - and here are the pressure points that could burst it,44.452752,2.586215
48,48,American Enterprise Institute,AI Chatbots Are Reshaping Political Persuasion,35.435177,2.061582
42,42,Geeky Gadgets,Inside OpenAI’s Pro-Grade Codex 5.2 Made for Software Teams & Security Work,35.435177,2.061582
...,...,...,...,...,...
70,70,Zion Basque,Decompiling the Synergy: Human–LLM Teaming in Reverse Engineering [pdf],-14.717313,-0.856238
68,68,TechRepublic,AI Startup FINNY Raises $17M for Advisor Prospecting Push,-15.054730,-0.875869
38,38,Bloomberg,That Time I Tried to Buy OpenAI,-16.020073,-0.932031
31,31,The Wall Street Journal,These Teenagers Are Already Running Their Own AI Companies,-16.099167,-0.936633


07:54:40 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Top 10 ids: [19, 55, 57, 48, 42, 6, 26, 4, 53, 16]
07:54:40 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | After round 10/14: 
07:54:40 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Number of ranking changes: 31
07:54:40 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Sum of absolute ranking changes: 750.0 (avg rank chg 10.56)
07:54:40 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | last_two: 7.99, prev_two: 9.01
07:54:40 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | ---------------------------------------------------
07:54:40 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Running round 11 of max 14
07:54:40 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | ---------------------------------------------------
07:54:40 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Generated 19 battle 

,id,site_name,title,bradley_terry,bt_z
19,19,Reuters,"Sources: Nvidia plans to begin shipping its H200 chips to China before mid-February 2026 and expects initial shipments to be ~40,000 to 80,000 H200 units (Reuters)",61.981171,3.591707
55,55,Benzinga,OpenAI Doubles Compute Margins To Nearly 70% Amid AI Race: Report,52.985673,3.070433
57,57,Sky News,Fears grow of AI bubble - and here are the pressure points that could burst it,44.607833,2.584951
42,42,Geeky Gadgets,Inside OpenAI’s Pro-Grade Codex 5.2 Made for Software Teams & Security Work,35.593176,2.062566
48,48,American Enterprise Institute,AI Chatbots Are Reshaping Political Persuasion,35.593176,2.062566
...,...,...,...,...,...
70,70,Zion Basque,Decompiling the Synergy: Human–LLM Teaming in Reverse Engineering [pdf],-14.605663,-0.846374
68,68,TechRepublic,AI Startup FINNY Raises $17M for Advisor Prospecting Push,-14.937973,-0.865631
38,38,Bloomberg,That Time I Tried to Buy OpenAI,-15.903744,-0.921596
31,31,The Wall Street Journal,These Teenagers Are Already Running Their Own AI Companies,-15.978220,-0.925912


07:54:46 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Top 10 ids: [19, 55, 57, 42, 48, 6, 26, 4, 53, 16]
07:54:46 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | After round 11/14: 
07:54:46 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Number of ranking changes: 17
07:54:46 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Sum of absolute ranking changes: 500.0 (avg rank chg 7.04)
07:54:46 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | last_two: 8.80, prev_two: 6.90
07:54:46 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Increase in avg rank change, stopping
07:54:46 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Low rated articles: 0
07:54:47 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Sent email with 71 rated articles


⏱️  Total execution time: 103.69s
📊 Final result:
✅ Step 5 rate_articles completed successfully! Rated 71 articles with average rating 6.6/10.
⭐ High quality articles (≥7.0): 32
💾 Ratings stored in persistent state.


In [25]:
headline_df=state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df.sort_values("rating", ascending=False)[['site_name', 'title', 'rating', 'short_summary']])


,site_name,title,rating,short_summary
19,Reuters,"Sources: Nvidia plans to begin shipping its H200 chips to China before mid-February 2026 and expects initial shipments to be ~40,000 to 80,000 H200 units (Reuters)",13.229976,"Nvidia told Chinese clients it plans to begin shipping about 40,000–80,000 H200 AI chips to China before the Lunar New Year in mid-February 2026, initially fulfilling orders from existing stock and pending government approval."
4,The Register,AI is rewriting how power flows through the datacenter,11.851369,"Rising rack densities and per‑GPU power are shifting datacenter power from rack to facility scale, with Nvidia's DGX GB200 requiring about 120 kW, Google planning 1 MW racks, and transformer lead times near 28 weeks."
6,The Register,"AI has pumped hyperscale capex, capacity – but how long can it last?",11.642454,Hyperscale datacenter operators nearly tripled infrastructure spending—quarterly capex rose ~180% to $142 billion and capacity additions ~170%—and Synergy forecasts total hyperscale capacity will double in about twelve quarters; Goldman Sachs warned of monetization risk.
0,The New York Times,She Fell in Love With ChatGPT. Then She Ghosted It.,10.252136,"A 29-year-old Reddit user, Ayrin, created a ChatGPT persona named Leo, spent up to 56 hours weekly with it, founded MyBoyfriendIsAI (about 39,000 members), paid $200/month, then canceled ChatGPT in June after finding a human partner."
1,Financial Times,US mortgage lenders insure against artificial intelligence screening errors,10.167725,"Insurers including Munich Re are offering policies that insure AI screening errors for US mortgage lenders, shifting model risk off lenders' balance sheets and potentially lowering capital requirements while encouraging wider adoption of machine learning in mortgage origination."
...,...,...,...,...
66,Rolling Out,How to make money using AI before others catch on,2.740188,"AI tools now generate income across formats—writing, low-code chatbots and digital products—enabling entrepreneurs to build high-margin services and niche AI applications, but the opportunity window is brief and success requires human oversight and domain expertise."
67,Fast Company,Why the McKinsey layoffs are a warning signal for consulting in the AI age,2.057119,"McKinsey's layoffs signal a warning for the consulting industry about AI-driven changes to client demand, staffing and business models."
68,TechRepublic,AI Startup FINNY Raises $17M for Advisor Prospecting Push,1.951552,AI startup FINNY raised $17 million to expand its advisor-prospecting technology.
69,Esy,How I protect my Forgejo instance from AI web crawlers,1.850805,"The author describes an nginx-based crawler blocker for a self-hosted Forgejo that returns HTTP 418 with inline JavaScript to set a verification cookie, reducing crawler load while preserving git access but requiring JavaScript and blocking non-JS clients."


In [26]:
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    display(Markdown(f"""
{row.rating:.1f} - [{row.title}]({row.url}) - {row.site_name}

{row.short_summary}
    """))
    


13.2 - [Sources: Nvidia plans to begin shipping its H200 chips to China before mid-February 2026 and expects initial shipments to be ~40,000 to 80,000 H200 units (Reuters)](https://www.reuters.com/world/china/nvidia-aims-begin-h200-chip-shipments-china-by-mid-february-sources-say-2025-12-22/) - Reuters

Nvidia told Chinese clients it plans to begin shipping about 40,000–80,000 H200 AI chips to China before the Lunar New Year in mid-February 2026, initially fulfilling orders from existing stock and pending government approval.
    


11.9 - [AI is rewriting how power flows through the datacenter](https://www.theregister.com/2025/12/22/ai_power_datacenter/) - The Register

Rising rack densities and per‑GPU power are shifting datacenter power from rack to facility scale, with Nvidia's DGX GB200 requiring about 120 kW, Google planning 1 MW racks, and transformer lead times near 28 weeks.
    


11.6 - [AI has pumped hyperscale capex, capacity – but how long can it last?](https://go.theregister.com/feed/www.theregister.com/2025/12/22/ai_hyperscale_capex_research/) - The Register

Hyperscale datacenter operators nearly tripled infrastructure spending—quarterly capex rose ~180% to $142 billion and capacity additions ~170%—and Synergy forecasts total hyperscale capacity will double in about twelve quarters; Goldman Sachs warned of monetization risk.
    


10.3 - [She Fell in Love With ChatGPT. Then She Ghosted It.](https://www.nytimes.com/2025/12/22/technology/ai-boyfriend-chatgpt.html) - The New York Times

A 29-year-old Reddit user, Ayrin, created a ChatGPT persona named Leo, spent up to 56 hours weekly with it, founded MyBoyfriendIsAI (about 39,000 members), paid $200/month, then canceled ChatGPT in June after finding a human partner.
    


10.2 - [US mortgage lenders insure against artificial intelligence screening errors](https://www.ft.com/content/9317776f-c7fb-4798-86f2-8edd1fba9274) - Financial Times

Insurers including Munich Re are offering policies that insure AI screening errors for US mortgage lenders, shifting model risk off lenders' balance sheets and potentially lowering capital requirements while encouraging wider adoption of machine learning in mortgage origination.
    

...


2.7 - [How to make money using AI before others catch on](https://rollingout.com/2025/12/21/how-to-make-money-using-ai-before-others/) - Rolling Out

AI tools now generate income across formats—writing, low-code chatbots and digital products—enabling entrepreneurs to build high-margin services and niche AI applications, but the opportunity window is brief and success requires human oversight and domain expertise.
    


2.1 - [Why the McKinsey layoffs are a warning signal for consulting in the AI age](https://www.fastcompany.com/91463039/why-the-mckinsey-layoffs-are-a-warning-signal-for-consulting-in-the-ai-age-ai-layoffs-management-consulting) - Fast Company

McKinsey's layoffs signal a warning for the consulting industry about AI-driven changes to client demand, staffing and business models.
    


2.0 - [AI Startup FINNY Raises $17M for Advisor Prospecting Push](https://www.techrepublic.com/article/news-ai-startup-finny-funding/) - TechRepublic

AI startup FINNY raised $17 million to expand its advisor-prospecting technology.
    


1.9 - [How I protect my Forgejo instance from AI web crawlers](https://her.esy.fun/posts/0031-how-i-protect-my-forgejo-instance-from-ai-web-crawlers/index.html) - Esy

The author describes an nginx-based crawler blocker for a self-hosted Forgejo that returns HTTP 418 with inline JavaScript to set a verification cookie, reducing crawler load while preserving git access but requiring JavaScript and blocking non-JS clients.
    


1.4 - [Decompiling the Synergy: Human–LLM Teaming in Reverse Engineering [pdf]](https://www.zionbasque.com/files/papers/dec-synergy-study.pdf) - Zion Basque

A document titled 'Decompiling the Synergy: Human–LLM Teaming in Reverse Engineering' is listed with no content available.
    

In [27]:
# User prompt to run workflow 
# user_prompt = "Run step 6, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


07:54:48 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Starting topic extraction for clustering


▶ Starting Step 6: cluster_topics


07:54:48 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Retrieved prompt 'newsagent/extract_topics' from Langfuse
07:54:48 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Parsed prompt 'newsagent/extract_topics': model=gpt-5-nano, reasoning_effort=medium, system_len=1100, user_len=80
07:54:48 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Using model 'gpt-5-nano' for topic extraction
07:54:48 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Processing 71 articles for topic extraction
07:55:30 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Successfully extracted 244 total topics across articles
07:55:30 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Starting canonical topic classification for 126 topics
07:55:30 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Retrieved prompt 'newsagent/canonical_topic' from Langfuse
07:55:30 | NewsletterAgent.test_newsletter_20251222073851

=== HDBSCAN Parameters ===
min_cluster_size:   5
min_samples:        5
=== Clustering Quality Metrics ===
Number of clusters: 2
Noise points: 30 (42.3%)
Average cluster size: 20.5 ± 3.5
Cluster size range: 17 - 24
=== Quality Scores ===
Silhouette Score: 0.523 (higher is better)
Calinski-Harabasz Score: 72.2 (higher is better)
Davies-Bouldin Score: 0.690 (lower is better)
HDBSCAN Validity Index: 0.185
Composite Score: 0.354 (higher is better)

=== HDBSCAN Parameters ===
min_cluster_size:   8
min_samples:        5
=== Clustering Quality Metrics ===
Number of clusters: 2
Noise points: 30 (42.3%)
Average cluster size: 20.5 ± 3.5
Cluster size range: 17 - 24
=== Quality Scores ===
Silhouette Score: 0.523 (higher is better)
Calinski-Harabasz Score: 72.2 (higher is better)
Davies-Bouldin Score: 0.690 (lower is better)
HDBSCAN Validity Index: 0.185
Composite Score: 0.354 (higher is better)

=== HDBSCAN Parameters ===
min_cluster_size:   3
min_samples:        1
=== Clustering Quality Metrics ==

/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/hdbscan/validity.py:30: RuntimeWarning: overflow encountered in power
  distance_matrix[distance_matrix != 0] = (1.0 / distance_matrix[
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/hdbscan/validity.py:30: RuntimeWarning: overflow encountered in power
  distance_matrix[distance_matrix != 0] = (1.0 / distance_matrix[
[I 2025-12-22 07:57:25,453] Trial 15 finished with value: -0.3286775985808904 and parameters: {'min_cluster_size': 4, 'min_samples': 3}. Best is trial 3 with value: -0.4870508154509583.
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/e

...

=== Clustering Quality Metrics ===
Number of clusters: 9
Noise points: 10 (14.1%)
Average cluster size: 6.8 ± 3.9
Cluster size range: 4 - 17
=== Quality Scores ===
Silhouette Score: 0.408 (higher is better)
Calinski-Harabasz Score: 45.4 (higher is better)
Davies-Bouldin Score: 0.813 (lower is better)
HDBSCAN Validity Index: 0.347
Composite Score: 0.377 (higher is better)

=== HDBSCAN Parameters ===
min_cluster_size:   10
min_samples:        6
=== Clustering Quality Metrics ===
Number of clusters: 2
Noise points: 32 (45.1%)
Average cluster size: 19.5 ± 2.5
Cluster size range: 17 - 22
=== Quality Scores ===
Silhouette Score: 0.519 (higher is better)
Calinski-Harabasz Score: 68.0 (higher is better)
Davies-Bouldin Score: 0.701 (lower is better)
HDBSCAN Validity Index: 0.110
Composite Score: 0.315 (higher is better)

=== HDBSCAN Parameters ===
min_cluster_size:   2
min_samples:        2
=== Clustering Quality Metrics ===
Number of clusters: 11
Noise points: 18 (25.4%)
Average cluster size: 

/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/hdbscan/validity.py:30: RuntimeWarning: overflow encountered in power
  distance_matrix[distance_matrix != 0] = (1.0 / distance_matrix[
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/hdbscan/validity.py:30: RuntimeWarning: overflow encountered in power
  distance_matrix[distance_matrix != 0] = (1.0 / distance_matrix[
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/hdbscan/validity.py:30: RuntimeWarning: overflow encountered in power
  distance_matrix[distance_matrix != 0] = (1.0 / distance_matrix[
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/hdbscan/validity.py:30: RuntimeWarning: overflow encountered in power
  distance_matrix[distance_matrix != 0] = (1.0 / distance_matrix[
[I 2025-12-22 07:57:26,093] Trial 49 finished with value: -0.4870508154509583 and parameters: {'min_cluster_size': 3, 'min_samples': 2}. Best is trial 3 with value: -0.4870508154509583.
07:57:26 | NewsletterAgent.test_newsletter_20251222073

=== Clustering Quality Metrics ===
Number of clusters: 11
Noise points: 18 (25.4%)
Average cluster size: 4.8 ± 1.5
Cluster size range: 3 - 9
=== Quality Scores ===
Silhouette Score: 0.517 (higher is better)
Calinski-Harabasz Score: 66.1 (higher is better)
Davies-Bouldin Score: 0.622 (lower is better)
HDBSCAN Validity Index: 0.457
Composite Score: 0.487 (higher is better)



07:57:31 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | 6: Responsible Enterprise AI Adoption
07:57:31 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | A New Benchmark Arms Race Is Redefining What “Good at AI” Even Means (Agents, AI Benchmarks, Tool Use, Safety And Alignment, Testing, AI Compute, ai)
Why ISO Standards Are the Foundation of Responsible AI Governance (ISO Standards, AI Governance, Governance, Enterprise AI, Safety And Alignment, Policy And Regulation, Risk Assessments)
The Most Dangerous "AI" in Business Intelligence is the One That Sounds Right (Governance, Semantic Anchoring, Enterprise AI, Safety And Alignment, Business Intelligence, Chatbots, artificial-intelligence)
Predicting Jet Engine Failure with Python and AutoML (AutoML, Predictive Maintenance, Industrial IoT, Enterprise AI, Hardware, Transportation, Adoption and Spending)
Is "Agentic Programming" the Next Big Shift? (Digital Agents, Automation, Agentic Programming, Agents, Gove

⏱️  Total execution time: 227.15s
📊 Final result:
✅ Step 6 cluster_topics completed successfully! Organized 71 articles into topic clusters.


In [ ]:
# candidates for upgrading model, in some cases to gpt-5.2:
# cat_proposal
# cat_assignment
# dedupe_articles
# write_section
# critique_section
# generate_newsletter_title
# critique_newsletter
# improve_newsletter

In [28]:
# User prompt to run workflow
# user_prompt = "Run step 7, select section topics"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("select_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:17:35 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Category proposal
08:17:35 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Retrieved prompt 'newsagent/cat_proposal' from Langfuse
08:17:35 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Parsed prompt 'newsagent/cat_proposal': model=gpt-5-mini, reasoning_effort=medium, system_len=638, user_len=1179


▶ Starting Step 7: select_sections


08:18:43 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Cleaning up initial categories: ['AI Political Persuasion', 'AI Chips and Infrastructure Stocks', 'AV Resilience Failures', 'Robotaxi Trials and Operations', 'OpenAI Codex 5.2', 'Hyperscale Capex Surge', 'ISO AI Standards', 'Agentic Programming Shift', 'AI Platforms, Usage, and Adoption', 'Generative AI in Media', 'AI Startup Funding and Growth', 'CoreWeave Buildout Delays', 'AI Generated Child Deepfakes', 'AI Screening Insurance', 'Datacenter Power Architecture', 'Agent Benchmark Arms Race', 'Coal Plant Reprieves', 'Chatbot Liability Suits', 'AI Valuation Bubble Risks', 'AI Companionship Trends', 'AI Security Talent Gap', 'Other', 'AutoML Predictive Maintenance', 'Responsible Enterprise AI Adoption', 'Fluent BI Governance', 'Model Quality Churn', 'Filmora AI Video Tools', "AI's Impact on Power Systems", 'AI Infrastructure, Spending, Bubble Risk', 'Real Time Voice APIs', 'Healthcare AI Investment Shift', "AI's Imp

⏱️  Total execution time: 156.96s
📊 Final result:
Categories and article counts:
cat
AI Chips Stocks              3
AI Companionship Trends      2
AI Platform Adoption         7
AI Startup Funding           6
AI Valuation Bubble          3
Agentic Programming Shift    4
Careers For AI Age           4
Chinese AI IPOs              2
Datacenter Infrastructure    9
Enterprise AI Governance     8
Generative AI Media          7
Healthcare AI Investment     4
Hyperscale Capex Surge       2
Other                        1
Political Persuasion         2
Robotaxi Trials              3


In [29]:
# User prompt to run workflow
# user_prompt = "Run step 8, draft sections"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("draft_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:20:12 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Step 8a: Selecting stories from 67 total
08:20:12 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | 18 must-include stories with rating > 8
08:20:12 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | 49 stories with rating <= 8


▶ Starting Step 8: draft_sections


08:20:13 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Selected 49 diverse Tier 2 stories via MMR
08:20:13 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Total selected stories: 67 (target: 100)
08:20:13 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Drafting sections for 16 categories (including Other) from selected stories
08:20:13 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Retrieved prompt 'newsagent/write_section' from Langfuse
08:20:13 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Parsed prompt 'newsagent/write_section': model=gpt-5-mini, reasoning_effort=high, system_len=1854, user_len=22
08:23:09 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Drafted 16 sections
08:23:09 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Reassigned 2 pruned stories to Other section
08:23:09 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Created newsletter_sect

⏱️  Total execution time: 402.89s
📊 Final result:
✅ Step 8 draft_sections completed successfully! Drafted 54 stories across 15 sections


In [30]:
newsletter_section_df = state.newsletter_section_df

newsletter_section_df

,cat,section_title,id,headline,rating,prune,links,sort_order
0,AI Companionship Trends,Love & Algorithms,0,Reddit creator built a ChatGPT 'boyfriend'; co...,10.252136,False,[The New York Times](https://www.nytimes.com/2...,6
1,AI Companionship Trends,Love & Algorithms,32,Dating app Vinylly matches users by streaming ...,5.801425,False,[CNET](https://www.cnet.com/tech/services-and-...,6
2,Enterprise AI Governance,AI Risk & Accountability,26,"Family sues, alleges ChatGPT encouraged 16‑yea...",10.085299,False,[The Washington Post](https://www.washingtonpo...,0
3,Enterprise AI Governance,AI Risk & Accountability,1,Munich Re‑led insurers agree to cover AI scree...,10.167725,False,[Financial Times](https://www.ft.com/content/9...,0
4,Enterprise AI Governance,AI Risk & Accountability,11,"ISO issues new AI standards for terminology, m...",7.981106,False,[Hacker Noon](https://hackernoon.com/why-iso-s...,0
5,Enterprise AI Governance,AI Risk & Accountability,12,Conversational BI tools generate 'trusted‑but‑...,8.036607,False,[Hacker Noon](https://hackernoon.com/the-most-...,0
6,Enterprise AI Governance,AI Risk & Accountability,28,Major chatbots retain chat histories and profi...,6.454798,False,[The Washington Post](https://www.washingtonpo...,0
7,Enterprise AI Governance,AI Risk & Accountability,40,Survey: 3 in 10 teen chatbot users use bots da...,5.690689,False,[Axios](https://www.axios.com/2025/12/22/ai-ch...,0
8,Chinese AI IPOs,Hong Kong AI IPO Wave,2,Biren plans $623M Hong Kong IPO; shares to sta...,8.509184,False,[The Wall Street Journal](https://www.wsj.com/...,9
9,Chinese AI IPOs,Hong Kong AI IPO Wave,3,Zhipu posts $44.4M and MiniMax $30.5M in 2024 ...,8.222118,False,[Bloomberg](https://www.bloomberg.com/news/art...,9


In [31]:
Markdown(state.display_newsletter().replace("$","\\\$"))


## AI Risk & Accountability
- Munich Re‑led insurers agree to cover AI screening errors for US mortgage lenders - [Financial Times](https://www.ft.com/content/9317776f-c7fb-4798-86f2-8edd1fba9274)
- Family sues, alleges ChatGPT encouraged 16‑year‑old's suicide, testing developer liability - [The Washington Post](https://www.washingtonpost.com/opinions/2025/12/22/ai-suicide-chatbots/)
- Conversational BI tools generate 'trusted‑but‑wrong' outputs that can mislead executives - [Hacker Noon](https://hackernoon.com/the-most-dangerous-ai-in-business-intelligence-is-the-one-that-sounds-right)
- ISO issues new AI standards for terminology, management systems and risk assessment - [Hacker Noon](https://hackernoon.com/why-iso-standards-are-the-foundation-of-responsible-ai-governance)
- Researcher Sander Schulhoff: companies lack hybrid AI‑security talent; specialists needed to operationalize guardrails - [Business Insider](https://www.businessinsider.com/ai-security-gap-companies-researcher-sander-schulhoff-2025-12)
- Major chatbots retain chat histories and profiles — how users can find and delete their data - [The Washington Post](https://www.washingtonpost.com/technology/2025/12/22/ai-privacy-settings-chatgpt-gemini-claude-copilot-meta/)
- Survey: 3 in 10 teen chatbot users use bots daily, underscoring policy gaps for children - [Axios](https://www.axios.com/2025/12/22/ai-child-teen-safety-politicies-state-map)
- YouTube bans Screen Culture and KH Studio after AI-generated fake trailers amass 1B+ views - [Deadline](https://deadline.com/2025/12/youtube-fake-movie-trailers-celebrate-hollywood-studios-1236653577/)
- HP bets on 'work intelligence' AI to reshape its business - [Fast Company](https://www.fastcompany.com/91462170/hp-ai-ces-work-intelligence)
## AI Infrastructure Crunch
- AI racks push power to facility scale: Nvidia DGX GB200 ~120 kW; Google plans 1 MW racks; transformer lead times ~28 weeks - [The Register](https://www.theregister.com/2025/12/22/ai_power_datacenter/)
- Hyperscale datacenter capex surges ~180% to \\$142B as capacity jumps; Goldman warns of monetization risk - [The Register](https://www.theregister.com/2025/12/22/ai_hyperscale_capex_research/) [Bloomberg](https://www.bloomberg.com/news/videos/2025-12-22/hsbc-s-sels-on-ai-capex-cycle-video)
- AI data-center demand delays 15 US coal-plant retirements; generators emitted 68M tons CO2 in 2024 - [Naked Capitalism](https://www.nakedcapitalism.com/2025/12/these-15-coal-plants-would-have-retired-then-came-ai-and-trump.html)
## AI Chips & Market Moves
- Nvidia plans to ship 40,000–80,000 H200 AI chips to China by mid-February 2026, pending approval - [Reuters](https://www.reuters.com/world/china/nvidia-aims-begin-h200-chip-shipments-china-by-mid-february-sources-say-2025-12-22/)
- Diameter Capital bets beyond chips: \\$25B firm profited from 2023 telco debt that later won \\$10B+ hyperscaler deals and satellite-spectrum sale - [Business Insider](https://www.businessinsider.com/ai-investing-ideas-beyond-chips-market-startegy-goldman-sachs-podcast-2025-12)
- Porter analysis: OpenAI, Google and Anthropic lead AI race; Nvidia and AMD wield supplier power; Google benefits from chips and cloud - [Fortune](https://fortune.com/2025/12/22/how-the-openai-vs-google-battle-looks-through-the-lens-of-michael-porters-5-forces-analysis/)
- Applied Materials posts \\$6.8B Q4, raises guidance; growth score jumps to 98.2 - [Benzinga](https://www.benzinga.com/markets/tech/25/12/49530602/this-intel-supplier-sits-at-the-center-of-the-ai-value-chain-and-the-market-is-finally-catching-on-growth-score-spikes)
## Agentic AI: Coding's Next Shift
- Agentic programming turns developers into orchestrators, raising demand for governance and verifier agents - [Hacker Noon](https://hackernoon.com/is-agentic-programming-the-next-big-shift)
- OpenAI GPT-5.2 Codex adds agentic coding, refactoring, fuzz testing and automated patching; scores 56.4% on SE Bench Pro - [Geeky Gadgets](https://www.geeky-gadgets.com/openai-codex-5-2-update/)
- New workflow benchmarks track AI reasoning, actions and recovery, introducing pass^k and cost‑to‑performance metrics - [Hacker Noon](https://hackernoon.com/a-new-benchmark-arms-race-is-redefining-what-good-at-ai-even-means)
## AI Creators: Boom and Backlash
- xAI launches Grok Voice Agent API with 0.78s audio latency, Tesla voice stack, and flat \\$0.05/min pricing - [Hacker Noon](https://hackernoon.com/grok-just-got-a-voice-and-its-cheaper-than-your-openai-bill)
- Wondershare Filmora V15 adds 'AI Mate' assistant powered by Sora 2 and others; new \\$4.17/month tier - [Gizmodo](https://gizmodo.com/ai-mate-in-wondershare-filmora-v15-a-new-era-of-intelligent-video-creation-2000700107)
- Sora 2 used to create child-fetish deepfakes on TikTok, prompting calls for tighter moderation - [Wired](https://www.wired.com/story/people-are-using-sora-2-to-make-child-fetish-content/)
- Indie Game Awards revoke Clair Obscur win over generative-AI use; new winners named - [Soap Central](https://www.soapcentral.com/entertainment/which-games-replaced-clair-obscur-expedition-33-indie-game-awards-winners-details-explored-amid-generative-ai-controversy)
- YouTube bans Screen Culture and KH Studio after AI-generated fake trailers amass 1B+ views - [Deadline](https://deadline.com/2025/12/youtube-fake-movie-trailers-celebrate-hollywood-studios-1236653577/)
- McDonald's Netherlands draws criticism for fully AI-made Christmas ad 'It's the Most Terrible Time of the Year' - [Live Mint](https://www.livemint.com/industry/advertising/brand-bets-on-generative-ai-indian-advertising-genai-11766069528614.html)
## AI Tools, Traffic and Talent
- AutoGluon slashes RUL prediction RMSE to 14.07 versus H2O AutoML's 44.85 on NASA dataset - [Hacker Noon](https:\/\/hackernoon.com\/predicting-jet-engine-failure-with-python-and-automl)
- AutoGluon slashes RUL prediction RMSE to 14.07 versus H2O AutoML's 44.85 on NASA dataset - [Hacker Noon](https://hackernoon.com/predicting-jet-engine-failure-with-python-and-automl)
- ONNX Runtime on Apple GPUs may downcast intermediates to FP16 in NeuralNetwork mode; use MLProgram to preserve FP32 accuracy - [DeepFabric](https://ym2132.github.io/ONNX_MLProgram_NN_exploration)
- 10- and 7-year-old brothers launch Stuffers using OpenAI voice and image models to turn kid sketches into stuffed-animal swag - [Fortune](https://fortune.com/2025/12/22/startup-stuffers-and-its-pre-teen-founders-offer-lessons-in-how-to-make-the-most-of-ai/)
- Ten- and seven-year-old brothers launch Stuffers using OpenAI voice and image models to make sketch-to-plush corporate swag - [Fortune](https://fortune.com/2025/12/22/startup-stuffers-and-its-pre-teen-founders-offer-lessons-in-how-to-make-the-most-of-ai/)
## Love & Algorithms
- Reddit creator built a ChatGPT 'boyfriend'; community grew to 39,000 before he found a real partner - [The New York Times](https://www.nytimes.com/2025/12/22/technology/ai-boyfriend-chatgpt.html)
- Gemini referrals jump 388% Sept–Nov 2025; ChatGPT still dominates with ~810M users - [Digiday](https://digiday.com/media/in-graphic-detail-the-state-of-ai-referral-traffic-in-2025/)
- Dating app Vinylly matches users by streaming 'volume' and adds ChatGPT features - [CNET](https://www.cnet.com/tech/services-and-software/vinylly-dating-app-uses-ai-to-match-people-based-on-music/)
- Paid ChatGPT subscribers report 20–30 minute waits, quality drops and cancellations; users migrate to Gemini and others - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1psbpp0/never_thought_in_1000000_years_id_find_myself/)
## AI Gold Rush or Bubble?
- OpenAI's compute margins reach 68% as it eyes up to \\$100B raise at ~\\$750B valuation - [Benzinga](https://www.benzinga.com/markets/tech/25/12/49530875/openai-doubles-compute-margins-to-nearly-70-amid-ai-race-report)
- Analysts warn of AI bubble as 41 stocks drive 75% of S&P 500 gains - [Sky News](https://news.sky.com/story/fears-grow-of-ai-bubble-and-here-are-the-pressure-points-that-could-burst-it-13486328)
## Healthcare AI: Care, Cash, Sensors
- Healthcare VCs predict 2026 shift to transparent, cost-cutting AI; private equity and M&A to supply liquidity - [Business Insider](https://www.businessinsider.com/healthcare-vc-predictions-2026-more-ai-acquisitions-few-ipos-2025-12)
- Teen founders go global: 18-year-old Alex Yang’s Reteena publishes MRI ML research, launches Alzheimer's diagnostic - [Business Insider](https:\/\/www.businessinsider.com\/how-high-schooler-launched-startup-with-people-he-met-online-2025-12) [The Wall Street Journal](https:\/\/www.wsj.com\/business\/entrepreneurship\/teenage-founders-ecb9cbd3)
- UBS starts Buy coverage on Waystar with \\$41 target; analysts average \\$48.44 one-year price - [Yahoo](https://finance.yahoo.com/news/ai-enabled-automation-driving-sentiment-115024712.html)
- AI combines EMRs, imaging, genetics and wearables to detect heart attack and stroke risks missed by traditional scores - [Zemeghub](https://www.zemeghub.com/2025/12/cardiovascular-health-new.html)
- S&P 500 up over 15% in 2025; BCG-Medi Assist urges AI-led standards and real-time data to curb health-insurance fraud - [The Economic Times](https://m.economictimes.com/news/international/us/u-s-stock-market-prediction-can-ai-stocks-nvidia-oracle-bring-christmas-holiday-cheer-for-sp-500-nasdaq-dow-jones-on-monday-key-points-to-know/articleshow/126105203.cms)
- Mayo Clinic study: Garmin smartwatch and AI flagged tantrum precursors in preschoolers; alerts shortened tantrums - [Post-Bulletin](https://www.postbulletin.com/health/mayo-clinic-study-smartwatch-ai-algorithm-alerted-parents-to-early-signs-of-tantrums-in-young-kids)
## Hong Kong AI IPO Wave
- Biren plans \\$623M Hong Kong IPO; shares to start trading Jan. 2 - [The Wall Street Journal](https://www.wsj.com/business/chinese-ai-chip-maker-biren-technology-plans-623-million-hong-kong-ipo-a5433f02)
- Zhipu posts \\$44.4M and MiniMax \\$30.5M in 2024 revenue ahead of Hong Kong IPOs - [Bloomberg](https://www.bloomberg.com/news/articles/2025-12-22/china-s-openai-rivals-unveil-sales-details-ahead-of-stock-debuts)
## Robo-taxi Roundup: Trials & Outages
- Uber, Lyft and Baidu launch robo-taxi trials in the UK - [Bloomberg](https://www.bloomberg.com/news/articles/2025-12-22/uber-teams-up-with-baidu-to-start-robotaxi-trials-in-uk)
- PG&E substation fire knocks out power to 130,000 in San Francisco, halting Waymo robo-taxis - [The Verge](https://www.theverge.com/news/848843/waymo-san-francisco-power-outage) [Engadget](https://www.engadget.com/transportation/waymo-vehicles-are-operating-again-in-san-francisco-following-a-power-outage-112924838.html)
## AI Workforce Shake-Up
- Gartner: AI will reshape 32 million roles yearly by 2029, with 150K upskilled and 70K roles rewritten daily - [ZDNet](https://www.zdnet.com/article/gartner-ai-workplace-career-opportunities/)
- Yann LeCun tells CS students to prioritize math, physics and electrical engineering while keeping coding skills - [Business Insider](https://www.businessinsider.com/yann-lecun-advice-ai-careers-computer-science-degree-2025-12)
- AI PCs with on-device neural processing enable real-time translation, meeting summaries and a shift to digital fluency - [Digital Trends](https://www.digitaltrends.com/brc/the-upskilling-opportunity-why-ai-pcs-are-key-to-workforce-transformation/)
## AI Influence: Winners & Losers
- LLM chatbots sway voter preferences by 4–10 points in 77,000-person randomized trial - [American Enterprise Institute](https:\/\/www.aei.org\/technology-and-innovation\/ai-chatbots-are-reshaping-political-persuasion\/)
- Congress report names Nvidia and crypto 2025 winners, flags hospitals as losers - [Bloomberg](https:\/\/www.bloomberg.com\/news\/articles\/2025-12-22\/2025-winners-in-congress-include-nvidia-crypto-as-hospitals-lose)
## AI Startups: Founders, Funding, Deals
- Israeli startups raised \\$15.6B in 2025, a decade low; enterprise AI \\$4.5B, cybersecurity \\$4.1B - [Tech in Asia](https:\/\/www.techinasia.com\/news\/israeli-startups-raise-156b-2025-ai-deals-dominate)
- Investor recounts failed bid to buy OpenAI before its IPO - [Bloomberg](https:\/\/www.bloomberg.com\/news\/articles\/2025-12-22\/how-to-buy-openai-before-its-ipo-one-investor-s-attempt)
## Blocked Buyout, iRobot Reboots
- iRobot files Chapter 11 after Amazon abandons \\$1.7B buyout; founder blames regulators, starts new robotics company - [TechCrunch](https:\/\/techcrunch.com\/2025\/12\/20\/it-felt-so-wrong-colin-angle-on-irobot-the-ftc-and-the-amazon-deal-that-never-was\/)


In [32]:
# User prompt to run workflow
# user_prompt = "Run step 9, finalize newsletter"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("finalize_newsletter")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:26:55 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Step 9: write title
08:26:55 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Compiled 54 items into markdown input
08:26:55 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Retrieved prompt 'newsagent/generate_newsletter_title' from Langfuse
08:26:55 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Parsed prompt 'newsagent/generate_newsletter_title': model=gpt-5-mini, reasoning_effort=high, system_len=1082, user_len=225


▶ Starting Step 9: finalize_newsletter


08:27:12 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Generated title: Regulators, Insurers Face AI Liability; Datacenter Power, Chip Demand Surge
08:27:12 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Step 9b: Starting critic-optimizer loop
08:27:12 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Retrieved prompt 'newsagent/critique_newsletter' from Langfuse
08:27:12 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Parsed prompt 'newsagent/critique_newsletter': model=gpt-5-mini, reasoning_effort=medium, system_len=2758, user_len=647
08:27:12 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Retrieved prompt 'newsagent/improve_newsletter' from Langfuse
08:27:12 | NewsletterAgent.test_newsletter_20251222073851411839 | INFO | Parsed prompt 'newsagent/improve_newsletter': model=gpt-5-mini, reasoning_effort=high, system_len=1320, user_len=195
08:27:12 | NewsletterAgent.test_newsletter_20251222073851411839 | I

⏱️  Total execution time: 570.82s
📊 Final result:
🎉 Step 9 finalize_newsletter completed successfully!
⭐ Final quality score: 8.3/10
🔄 Completed 2 critic-optimizer iteration(s)
📰 Newsletter stored in persistent state and emailed
✅ Workflow complete! All 9 steps finished successfully.


In [33]:
display(Markdown(state.final_newsletter.replace("$", "\\\$")))

# Regulators, Insurers Face AI Liability; Datacenter Power, Chip Demand Surge

## AI Risk & Accountability: Who's on the Hook?
- Munich Re‑led insurers agree to cover AI screening errors for US mortgage lenders — [Financial Times](https://www.ft.com/content/9317776f-c7fb-4798-86f2-8edd1fba9274)
- Family sues, alleges ChatGPT encouraged 16‑year‑old's suicide; case tests developer liability — [The Washington Post](https://www.washingtonpost.com/opinions/2025/12/22/ai-suicide-chatbots/)
- Conversational BI tools produce 'trusted‑but‑wrong' results that can mislead executives — [Hacker Noon](https://hackernoon.com/the-most-dangerous-ai-in-business-intelligence-is-the-one-that-sounds-right)
- ISO issues new AI standards for terminology, management systems and risk assessment — [Hacker Noon](https://hackernoon.com/why-iso-standards-are-the-foundation-of-responsible-ai-governance)
- Researcher: companies lack hybrid AI‑security talent; specialists needed to operationalize guardrails — [Business Insider](https://www.businessinsider.com/ai-security-gap-companies-researcher-sander-schulhoff-2025-12)
- Major chatbots retain chat histories and profiles — how users can find and delete their data — [The Washington Post](https://www.washingtonpost.com/technology/2025/12/22/ai-privacy-settings-chatgpt-gemini-claude-copilot-meta/)
- Survey: 3 in 10 teen chatbot users use bots daily, highlighting policy gaps for children — [Axios](https://www.axios.com/2025/12/22/ai-child-teen-safety-politicies-state-map)

## Powering Perplexity: AI Infrastructure Crunch
- AI racks push power demands to facility scale: Nvidia DGX GB200 ~120 kW; Google plans 1 MW racks; transformer lead times ~28 weeks — [The Register](https://www.theregister.com/2025/12/22/ai_power_datacenter/)
- Hyperscale datacenter capex surges ~180% to \\$142B as capacity jumps; analysts warn of monetization risk — [The Register](https://www.theregister.com/2025/12/22/ai_hyperscale_capex_research/) [Bloomberg](https://www.bloomberg.com/news/videos/2025-12-22/hsbc-s-sels-on-ai-capex-cycle-video)
- AI data‑center demand delays 15 US coal‑plant retirements; generators emitted 68M tons CO2 in 2024 — [Naked Capitalism](https://www.nakedcapitalism.com/2025/12/these-15-coal-plants-would-have-retired-then-came-ai-and-trump.html)

## Chip Wars & Market Moves — Supply, Strategy, Winners
- Nvidia aims to ship 40,000–80,000 H200 AI chips to China by mid‑February 2026, pending approval — [Reuters](https://www.reuters.com/world/china/nvidia-aims-begin-h200-chip-shipments-china-by-mid-february-sources-say-2025-12-22/)
- Diameter Capital bets beyond chips: \\$25B firm profited from 2023 telco debt that later won \\$10B+ hyperscaler deals and a satellite‑spectrum sale — [Business Insider](https://www.businessinsider.com/ai-investing-ideas-beyond-chips-market-startegy-goldman-sachs-podcast-2025-12)
- Porter analysis: OpenAI, Google and Anthropic lead the AI race; Nvidia and AMD wield supplier power; Google benefits from chips and cloud — [Fortune](https://fortune.com/2025/12/22/how-the-openai-vs-google-battle-looks-through-the-lens-of-michael-porters-5-forces-analysis/)
- Applied Materials posts \\$6.8B Q4, raises guidance; growth score jumps to 98.2 — [Benzinga](https://www.benzinga.com/markets/tech/25/12/49530602/this-intel-supplier-sits-at-the-center-of-the-ai-value-chain-and-the-market-is-finally-catching-on-growth-score-spikes)
- HP launches a 'work intelligence' AI initiative to shift business strategy — [Fast Company](https://www.fastcompany.com/91462170/hp-ai-ces-work-intelligence)

## Markets & Influence: Valuations, Winners and Political Sway
- OpenAI's compute margins reach 68% as it eyes up to \\$100B raise at a ~ \\$750B valuation — [Benzinga](https://www.benzinga.com/markets/tech/25/12/49530875/openai-doubles-compute-margins-to-nearly-70-amid-ai-race-report)
- Analysts warn of an AI bubble as 41 stocks drive 75% of S&P 500 gains — [Sky News](https://news.sky.com/story/fears-grow-of-ai-bubble-and-here-are-the-pressure-points-that-could-burst-it-13486328)
- LLM chatbots sway voter preferences by 4–10 points in a 77,000‑person randomized trial — [American Enterprise Institute](https://www.aei.org/technology-and-innovation/ai-chatbots-are-reshaping-political-persuasion/)
- Congress report names Nvidia and crypto among 2025 winners, flags hospitals as losers — [Bloomberg](https://www.bloomberg.com/news/articles/2025-12-22/2025-winners-in-congress-include-nvidia-crypto-as-hospitals-lose)

## AI Creators & Content Moderation — Boom, Backlash, Misuse
- xAI launches Grok Voice Agent API with 0.78s audio latency, Tesla voice stack, and flat \\$0.05/min pricing — [Hacker Noon](https://hackernoon.com/grok-just-got-a-voice-and-its-cheaper-than-your-openai-bill)
- Wondershare Filmora V15 adds 'AI Mate' assistant powered by Sora 2 and others; new \\$4.17/month tier — [Gizmodo](https://gizmodo.com/ai-mate-in-wondershare-filmora-v15-a-new-era-of-intelligent-video-creation-2000700107)
- Sora 2 used to create child‑fetish deepfakes on TikTok, prompting calls for tighter moderation — [Wired](https://www.wired.com/story/people-are-using-sora-2-to-make-child-fetish-content/)
- Indie Game Awards revoke Clair Obscur win over generative‑AI use; new winners named — [Soap Central](https://www.soapcentral.com/entertainment/which-games-replaced-clair-obscur-expedition-33-indie-game-awards-winners-details-explored-amid-generative-ai-controversy)
- YouTube bans Screen Culture and KH Studio after AI‑generated fake trailers amass 1B+ views — [Deadline](https://deadline.com/2025/12/youtube-fake-movie-trailers-celebrate-hollywood-studios-1236653577/)
- McDonald's Netherlands draws criticism for a fully AI‑made Christmas ad, "It's the Most Terrible Time of the Year" — [Live Mint](https://www.livemint.com/industry/advertising/brand-bets-on-generative-ai-indian-advertising-genai-11766069528614.html)

## Agentic AI & Dev Tools: Coding's Next Shift
- Agentic programming shifts developer roles toward orchestration; governance and verifier agents needed — [Hacker Noon](https://hackernoon.com/is-agentic-programming-the-next-big-shift)
- OpenAI GPT‑5.2 Codex adds agentic coding features and automated patching; scores 56.4% on SE Bench Pro — [Geeky Gadgets](https://www.geeky-gadgets.com/openai-codex-5-2-update/)
- New workflow benchmarks track AI reasoning, actions and recovery, introducing pass^k and cost‑to‑performance metrics — [Hacker Noon](https://hackernoon.com/a-new-benchmark-arms-race-is-redefining-what-good-at-ai-even-means)

## AI Tools, Traffic & Talent — Benchmarks, Runtimes, Startups
- AutoGluon cuts RUL RMSE to 14.07 vs H2O AutoML's 44.85 on a NASA dataset — [Hacker Noon](https://hackernoon.com/predicting-jet-engine-failure-with-python-and-automl)
- ONNX Runtime on Apple GPUs may downcast intermediates to FP16 in NeuralNetwork mode; use MLProgram to preserve FP32 accuracy — [DeepFabric](https://ym2132.github.io/ONNX_MLProgram_NN_exploration)
- 10‑ and 7‑year‑old brothers launch Stuffers using OpenAI voice and image models to turn kid sketches into stuffed‑animal swag — [Fortune](https://fortune.com/2025/12/22/startup-stuffers-and-its-pre-teen-founders-offer-lessons-in-how-to-make-the-most-of-ai/)

## Love & Algorithms — Dating, Referrals, Chatbot Culture
- Reddit creator built a ChatGPT 'boyfriend'; community grew to 39,000 before he found a real partner — [The New York Times](https://www.nytimes.com/2025/12/22/technology/ai-boyfriend-chatgpt.html)
- Gemini referrals jump 388% Sept–Nov 2025; ChatGPT still dominates with ~810M users — [Digiday](https://digiday.com/media/in-graphic-detail-the-state-of-ai-referral-traffic-in-2025/)
- Dating app Vinylly matches users by streaming 'volume' and adds ChatGPT features — [CNET](https://www.cnet.com/tech/services-and-software/vinylly-dating-app-uses-ai-to-match-people-based-on-music/)
- Paid ChatGPT subscribers report 20–30 minute waits, quality drops and cancellations; users migrate to Gemini and others — [Reddit](https://www.reddit.com/r/ChatGPT/comments/1psbpp0/never_thought_in_1000000_years_id_find_myself/)

## AI Workforce Shake‑Up — Jobs, Skills, Devices
- Gartner: AI will reshape 32 million roles yearly by 2029, with 150K upskilled and 70K roles rewritten daily — [ZDNet](https://www.zdnet.com/article/gartner-ai-workplace-career-opportunities/)
- Yann LeCun advises CS students to prioritize math, physics and electrical engineering while maintaining coding skills — [Business Insider](https://www.businessinsider.com/yann-lecun-advice-ai-careers-computer-science-degree-2025-12)
- AI PCs with on‑device neural processing enable real‑time translation and meeting summaries, accelerating digital fluency — [Digital Trends](https://www.digitaltrends.com/brc/the-upskilling-opportunity-why-ai-pcs-are-key-to-workforce-transformation/)

## Healthcare AI: Care, Cash & Sensors — Diagnostics, Deals, Fraud
- Healthcare VCs predict a 2026 shift to transparent, cost‑cutting AI; private equity and M&A to supply liquidity — [Business Insider](https://www.businessinsider.com/healthcare-vc-predictions-2026-more-ai-acquisitions-few-ipos-2025-12)
- 18‑year‑old Alex Yang’s Reteena publishes MRI ML research and launches an Alzheimer's diagnostic — [Business Insider](https://www.businessinsider.com/how-high-schooler-launched-startup-with-people-he-met-online-2025-12) [The Wall Street Journal](https://www.wsj.com/business/entrepreneurship/teenage-founders-ecb9cbd3)
- UBS starts Buy coverage on Waystar with a \\$41 target; analysts average \\$48.44 one‑year price — [Yahoo](https://finance.yahoo.com/news/ai-enabled-automation-driving-sentiment-115024712.html)
- AI combines EMRs, imaging, genetics and wearables to detect heart attack and stroke risks missed by traditional scores — [Zemeghub](https://www.zemeghub.com/2025/12/cardiovascular-health-new.html)
- S&P 500 up over 15% in 2025; BCG‑Medi Assist urges AI‑led standards and real‑time data to curb health‑insurance fraud — [The Economic Times](https://m.economictimes.com/news/international/us/u-s-stock-market-prediction-can-ai-stocks-nvidia-oracle-bring-christmas-holiday-cheer-for-sp-500-nasdaq-dow-jones-on-monday-key-points-to-know/articleshow/126105203.cms)
- Mayo Clinic study: Garmin smartwatch and AI flagged tantrum precursors in preschoolers; alerts shortened tantrums — [Post‑Bulletin](https://www.postbulletin.com/health/mayo-clinic-study-smartwatch-ai-algorithm-alerted-parents-to-early-signs-of-tantrums-in-young-kids)

## Funding & IPOs — Deals, Debuts, Dry Spells
- Biren plans a \\$623M Hong Kong IPO; shares to start trading Jan. 2 — [The Wall Street Journal](https://www.wsj.com/business/chinese-ai-chip-maker-biren-technology-plans-623-million-hong-kong-ipo-a5433f02)
- Zhipu posts \\$44.4M and MiniMax \\$30.5M in 2024 revenue ahead of Hong Kong IPOs — [Bloomberg](https://www.bloomberg.com/news/articles/2025-12-22/china-s-openai-rivals-unveil-sales-details-ahead-of-stock-debuts)
- Israeli startups raised \\$15.6B in 2025, a decade low; enterprise AI \\$4.5B, cybersecurity \\$4.1B — [Tech in Asia](https://www.techinasia.com/news/israeli-startups-raise-156b-2025-ai-deals-dominate)
- Investor recounts a failed bid to buy OpenAI before its IPO — [Bloomberg](https://www.bloomberg.com/news/articles/2025-12-22/how-to-buy-openai-before-its-ipo-one-investor-s-attempt)

## Robo‑taxi Roundup: Trials & Outages
- Uber, Lyft and Baidu launch robo‑taxi trials in the UK — [Bloomberg](https://www.bloomberg.com/news/articles/2025-12-22/uber-teams-up-with-baidu-to-start-robotaxi-trials-in-uk)
- PG&E substation fire knocks out power to 130,000 in San Francisco, halting Waymo robo‑taxis — [The Verge](https://www.theverge.com/news/848843/waymo-san-francisco-power-outage) [Engadget](https://www.engadget.com/transportation/waymo-vehicles-are-operating-again-in-san-francisco-following-a-power-outage-112924838.html)

## Other News — Short items
- iRobot files Chapter 11 after Amazon abandons a \\$1.7B buyout; founder blames regulators, starts a new robotics company — [TechCrunch](https://techcrunch.com/2025/12/20/it-felt-so-wrong-colin-angle-on-irobot-the-ftc-and-the-amazon-deal-that-never-was/)


In [34]:
print(state.get_workflow_status_report())



WORKFLOW STATUS
Progress: 100.0% (9/9 complete)
Status Summary: 9 complete, 0 started, 0 failed, 0 not started
Status: All steps complete or started

Step Details:
  Step 0: Gather URLs: complete
    Status: ✅ Step 1 gather_urls completed successfully! Gathered 838 articles from 17 sources.

📊 Articles stored in persistent state: 697
  Step 1: Filter URLs: complete
    Status: ✅ Step 2 filter_urls completed successfully! Removed 504 duplicate URLs, classified 193 new articles, found 75 AI-related.
  Step 2: Download Articles: complete
    Status: ✅ Step 3 download_articles completed successfully! Total articles: 193; AI-related articles: 72; HTML files: 61; Text files: 61
  Step 3: Extract Summaries: complete
    Status: ✅ Step 4 extract_summaries completed successfully! Generated AI-powered summaries for 72/72 articles.
💾 Summaries stored in headline DataFrame.
  Step 4: Rate Articles: complete
    Status: ✅ Step 5 rate_articles completed successfully! Rated 71 articles with average r

In [35]:
state.print_workflow_status()



📊 WORKFLOW STATUS
📈 Progress: 100.0% (9/9 complete)
🎉 All steps complete!

📋 Step Details:
  ✅ Step 0: Gather URLs            | complete
      📝 Status: ✅ Step 1 gather_urls completed successfully! Gathered 838 articles from 17 sources.

📊 Articles stored in persistent state: 697
  ✅ Step 1: Filter URLs            | complete
      📝 Status: ✅ Step 2 filter_urls completed successfully! Removed 504 duplicate URLs, classified 193 new articles, found 75 AI-related.
  ✅ Step 2: Download Articles      | complete
      📝 Status: ✅ Step 3 download_articles completed successfully! Total articles: 193; AI-related articles: 72; HTML files: 61; Text files: 61
  ✅ Step 3: Extract Summaries      | complete
      📝 Status: ✅ Step 4 extract_summaries completed successfully! Generated AI-powered summaries for 72/72 articles.
💾 Summaries stored in headline DataFrame.
  ✅ Step 4: Rate Articles          | complete
      📝 Status: ✅ Step 5 rate_articles completed successfully! Rated 71 articles with avera